# Adding a Model Artifact

In this notebook, we present the steps for onboarding a model with its model artifact.  When Fiddler is provided with your real model artifact, it can produce high-fidelity explanations.  In contrast, models within Fiddler that use a surrogate model or no model artifact at all provide approximative explainability or no explainability at all.

Fiddler is the pioneer in enterprise Model Performance Management (MPM), offering a unified platform that enables Data Science, MLOps, Risk, Compliance, Analytics, and LOB teams to **monitor, explain, analyze, and improve ML deployments at enterprise scale**. 
Obtain contextual insights at any stage of the ML lifecycle, improve predictions, increase transparency and fairness, and optimize business revenue.

---

You can experience Fiddler's NLP monitoring ***in minutes*** by following these five quick steps:


1. Connect to Fiddler
2. Define Your Model Specifications
3. Add Your Model
4. Upload The Model Package 
5. Publish Production Events and Get Insights (including high-fidelity explainability, or XAI!)

# 0. Imports

In [35]:
!pip install -q fiddler-client

import fiddler as fdl
import pandas as pd
import yaml
import datetime
import time
from IPython.display import clear_output

print(f"Running Fiddler client version {fdl.__version__}")


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Running Fiddler client version 3.1.0.dev1


# 1. Connect to Fiddler

Before you can add information about your model with Fiddler, you'll need to connect using our Python client.

---

**We need a few pieces of information to get started.**
1. The URL you're using to connect to Fiddler
3. Your authorization token

The latter two of these can be found by pointing your browser to your Fiddler URL and navigating to the **Settings** page.

In [36]:
URL = 'https://prepred.dev.fiddler.ai'  # Make sure to include the full URL (including https://).
TOKEN = 'doV4TS71P1MbC2WkxV4pHMAL3UjOzggtEQyShOGYCpc'

Now just run the following code block to connect the client to your Fiddler environment.

In [37]:
fdl.init(
    url=URL,
    token=TOKEN)

Once you connect, you can create a new project by specifying a unique project ID in the client's [create_project](https://docs.fiddler.ai/reference/clientcreate_project) function.

In [38]:
PROJECT_NAME = 'simple_model_artifact_upload1'

project = fdl.Project(
    name=PROJECT_NAME)

project.create()

# 2. Define your model specifications

In this example, we'll be considering the case where we're a bank and we have **a model that predicts credit approval worthiness**.
  
In order to get insights into the model's performance, **Fiddler needs a small  sample of data that can serve as a baseline** for making comparisons with data in production.


---


*For more information on how to design a sample dataset, [click here](https://docs.fiddler.ai/docs/designing-a-baseline-dataset).*

In [39]:
PATH_TO_SAMPLE_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/churn_data_sample.csv'

sample_df = pd.read_csv(PATH_TO_SAMPLE_CSV)
sample_df

,customer_id,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,predicted_churn,churn,timestamp
0,27acd1c2,545,Texas,Male,37,9,110483.86,1,1,1,127394.67,0.897202,yes,1710428231855
1,27b36d0c,497,Texas,Female,55,7,131778.66,1,1,1,9972.64,0.997441,yes,1710428262096
2,27b5360a,509,New York,Female,29,0,107712.57,2,1,1,92898.17,0.920563,yes,1710428292338
3,27b5d650,743,Hawaii,Nonbinary,39,6,0.00,2,1,0,44265.28,0.779282,yes,1710428322579
4,27b236a8,699,Florida,Female,25,8,0.00,2,1,1,52404.47,0.825474,yes,1710428352821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,27b409ba,686,Texas,Male,39,3,129626.19,2,1,1,103220.56,0.760645,yes,1711032910888
19996,27aaff96,446,Massachusetts,Female,45,10,125191.69,1,1,1,128260.86,0.216093,no,1711032941130
19997,27ad3162,794,California,Male,35,6,0.00,2,1,1,68730.91,0.982021,yes,1711032971371
19998,27b076ce,832,California,Male,61,2,0.00,1,0,1,127804.66,0.071598,no,1711033001613


### 2.a Define Model Specification
In order to add your model to Fiddler, simply create a ModelSpec object with information about what each column of your data sample should used for.

Fiddler supports four column types:
1. **Inputs**
2. **Outputs** (Model predictions)
3. **Targets** (Ground truth values)
4. **Metadata**

In [40]:
model_spec = fdl.ModelSpec(
    inputs=[
        'creditscore',
        'geography',
        'gender',
        'age',
        'tenure',
        'balance',
        'numofproducts',
        'hascrcard',
        'isactivemember',
        'estimatedsalary'
    ],
    outputs=['predicted_churn'],
    targets=['churn'],
    metadata=['customer_id', 'timestamp']
)

If you have columns in your ModelSpec that denote prediction IDs or timestamps, then Fiddler can use these to power its analytics accordingly. Let's define those as well:

In [41]:
id_column = 'customer_id'
timestamp_column = 'timestamp'

### 2.b Define Model Task

Fiddler supports a variety of model tasks. In this case, we're adding a binary classification model.

For this, we'll create a ModelTask object and an additional ModelTaskParams object to specify the ordering of our positive and negative labels.

For a detailed breakdown of all supported model tasks, click here.

In [42]:
model_task = fdl.ModelTask.BINARY_CLASSIFICATION

task_params = fdl.ModelTaskParams(target_class_order=['no', 'yes'])

## 3. Add your model

Create a Model object and publish it to Fiddler, passing in:
1. Your data sample
2. Your ModelSpec object
3. Your ModelTask and ModelTaskParams objects
4. Your ID and timestamp columns

In [43]:
MODEL_NAME = 'bank_churn'

model = fdl.Model.from_data(
    name=MODEL_NAME,
    project_id=fdl.Project.from_name(PROJECT_NAME).id,
    source=sample_df,
    spec=model_spec,
    task=model_task,
    task_params=task_params,
    event_id_col=id_column,
    event_ts_col=timestamp_column
)

model.create()

## 4. Upload your model package

Now it's time to upload your model package to Fiddler.  To complete this step, we need to ensure we have 2 assets in a directory.  It doesn't matter what this directory is called, but for this example we will call it **/model**.

In [46]:
import os
os.makedirs("model")

***Your model package directory will need to contain:***
1. A **package.py** file which explains to Fiddler how to invoke your model's prediction endpoint
2. And the **model artifact** itself
3. A **requirements.txt** specifying which python libraries need by package.py

### 4.a Create the **package.py** file

The contents of the cell below will be written into our ***package.py*** file.  This is the step that will be most unique based on model type, framework and use case.  The model's ***package.py*** file also allows for preprocessing transformations and other processing before the model's prediction endpoint is called.  For more information about how to create the ***package.py*** file for a variety of model tasks and frameworks, please reference the [Uploading a Model Artifact](https://docs.fiddler.ai/docs/uploading-a-model-artifact#packagepy-script) section of the Fiddler product documentation.

In [47]:
%%writefile model/package.py

import pandas as pd
from pathlib import Path
import os
from sklearn.ensemble import RandomForestClassifier
import pickle as pkl

 
PACKAGE_PATH = Path(__file__).parent
TARGET = 'churn'
PREDICTION = 'predicted_churn'

class Random_Forest:


    def __init__(self, model_path, output_column=None):
        """
        :param model_path: The directory where the model is saved.
        :param output_column: list of column name(s) for the output.
        """
        self.model_path = model_path
        self.output_column = output_column
        
       
        file_path = os.path.join(self.model_path, 'model.pkl')
        with open(file_path, 'rb') as file:
            self.model = pkl.load(file)
    
    
    def predict(self, input_df):
        return pd.DataFrame(
            self.model.predict_proba(input_df.loc[:, input_df.columns != TARGET])[:,1], 
            columns=self.output_column)
    

def get_model():
    return Random_Forest(model_path=PACKAGE_PATH, output_column=[PREDICTION])

Writing model/package.py


### 4.b  Ensure your model's artifact is in the **/model** directory

Make sure your model artifact (*e.g. the model_unfair.pkl file*) is also present in the model package directory.  The following cell will move this model's pkl file into our */model* directory.

In [48]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/models/model.pkl", "model/model.pkl")
urllib.request.urlretrieve("https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/models/requirements.txt", "model/requirements.txt")

('model/requirements.txt', <http.client.HTTPMessage at 0x132eab490>)

### 4.c Upload Sample Dataset 

In [49]:
DATASET_NAME = 'baseline' 
#Publishing Pre-prod dataset
model.publish(
  source=sample_df, # we are using the same sample dataset we used earlier in model onboarding
  environment=fdl.EnvType.PRE_PRODUCTION, #environemnt parameter is used to designate this dataset as a pre-production data
  dataset_name=DATASET_NAME)

### 4.d Define Model Parameters and Upload Model Files

In [50]:
MODEL_DIR = 'model/'
DEPLOYMENT_PARAMS = {'memory': 1024, 'cpu': 1000}
model.add_artifact(
  model_dir=MODEL_DIR,
  deployment_params=fdl.DeploymentParams(**DEPLOYMENT_PARAMS)
)

Within your Fiddler environment's UI, you should now be able to see the newly created model.

# 5. Publish Production Events and Get Insights (including high-fidelity explainability, or XAI!)

**You're all done!**
  
Now just head to your Fiddler environment's UI and explore the model's fairness metrics by navigating to the model selecting the fairness tab on the top right.


In [51]:
PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/churn_events.csv'

production_df = pd.read_csv(PATH_TO_EVENTS_CSV)
# Shift the timestamps of the production events to be as recent as today 
production_df['timestamp'] = production_df['timestamp'] + (int(time.time() * 1000) - production_df['timestamp'].max())

In [52]:
dataset = model.publish(
  source=production_df, 
  environment=fdl.EnvType.PRODUCTION) #make sure to specify this environment as PRODUCTION 



---


**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.

If you're still looking for answers, fill out a ticket on [our support page](https://fiddlerlabs.zendesk.com/) and we'll get back to you shortly.